In [35]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
loader = TextLoader("sample.txt", encoding="utf-8")
docs = loader.load()

In [25]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=60)
chunks = splitter.split_documents(docs)

In [26]:
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")
vector_db = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vector_db.as_retriever()

In [30]:
prompt = PromptTemplate.from_template("""Here is the context of the document: {context}\n
You are a question answering agent, answer the following question: {question}
""")
llm = ChatOllama(model="gemma3:4b")

In [33]:
from langchain_core.runnables import RunnablePassthrough

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [34]:
chain.invoke("Summarize the document")

AIMessage(content='The document advises against using biting words and emphasizes controlling what you reveal to others. It suggests that revealing too little through short answers and silence will frustrate others, causing them to fill the silence with unwanted comments and reveal their own intentions.', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-10-14T04:26:58.4090668Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12068232000, 'load_duration': 10749997600, 'prompt_eval_count': 248, 'prompt_eval_duration': 297984000, 'eval_count': 47, 'eval_duration': 961909000, 'model_name': 'gemma3:4b'}, id='run--4d1da02a-73b7-4545-a4f9-40e5250835fc-0', usage_metadata={'input_tokens': 248, 'output_tokens': 47, 'total_tokens': 295})